Environment setup.

In [ ]:
!pip install datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
colab_data_path = "/content/drive/MyDrive/Seminar2/data/ribe_512x768/"
colab_dir = "/content/drive/MyDrive/Seminar2/model/"
model_name_or_path = "google/vit-base-patch16-224-in21k"

Imports.

In [2]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.optimizers import *
from tensorflow.keras.losses import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.preprocessing.image import *
from tensorflow.keras.utils import *
from tensorflow.keras import * 
from sklearn.model_selection import train_test_split
from torchvision import transforms
from datasets import load_dataset, concatenate_datasets
import numpy as np
from sklearn import metrics

Using Huggingface datasets library to prepare the dataset splits.

In [ ]:
dataset = load_dataset("imagefolder", data_dir=colab_data_path)
splits = dataset["train"].train_test_split(test_size=0.33)
dataset["train"] = splits["train"]
dataset["val"] = splits["test"]

Define image augmentations.

In [6]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
    transforms.RandomRotation(30),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.8, 1.2)),
])

Apply the transforms to the train set. Concatenate the transformed and original train set.

In [7]:
transformed_train_dataset = dataset["train"].map(
    lambda example: {"image": transform(example["image"]), "label": example["label"]}
)

dataset["train"] = concatenate_datasets([transformed_train_dataset, dataset["train"]])

Map:   0%|          | 0/892 [00:00<?, ? examples/s]

Convert the dataset to a format suitable for Keras ResNet50.

In [ ]:
X_train = np.array(dataset["train"]["image"])
X_train = np.array([tf.convert_to_tensor(np.array(image)) for image in X_train])
y_train = to_categorical(np.array(dataset["train"]["label"]), num_classes=3)

X_valid = np.array(dataset["val"]["image"])
X_valid = np.array([tf.convert_to_tensor(np.array(image)) for image in X_valid])
y_valid = to_categorical(np.array(dataset["val"]["label"]), num_classes=3)

X_test = np.array(dataset["test"]["image"])
X_test = np.array([tf.convert_to_tensor(np.array(image)) for image in X_test])
y_test = to_categorical(np.array(dataset["test"]["label"]), num_classes=3)

Free unused variables.

In [28]:
del dataset
del splits
del transformed_train_dataset

Build the base model.

In [29]:
base_model= ResNet50(input_shape=(512,768,3), weights='imagenet', include_top=False)
model=Sequential()
model.add(base_model)
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(256,kernel_initializer='he_uniform', kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax', kernel_regularizer=regularizers.l2(0.001)))

for layer in base_model.layers:
    layer.trainable = False

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'],
)

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 16, 24, 2048)      23587712  
                                                                 
 flatten_1 (Flatten)         (None, 786432)            0         
                                                                 
 batch_normalization_2 (Batc  (None, 786432)           3145728   
 hNormalization)                                                 
                                                                 
 dense_2 (Dense)             (None, 256)               201326848 
                                                                 
 batch_normalization_3 (Batc  (None, 256)              1024      
 hNormalization)                                                 
                                                                 
 activation_1 (Activation)   (None, 256)              

Configure callbacks.

In [30]:
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='/content/drive/MyDrive/Seminar2/resnet50/',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

es = EarlyStopping(
    monitor='val_accuracy', 
    mode='max',
    patience=6
)

Train the model.

In [31]:
model.fit(
    x=X_train,
    y=y_train,
    batch_size=4,
    steps_per_epoch=50,
    epochs=30,
    validation_data=(X_valid, y_valid),
    validation_steps=25,
    callbacks=[es, checkpoint_callback],
    verbose=1
)

Epoch 1/30
49/50 [============================>.] - ETA: 0s - loss: 4.5788 - accuracy: 0.5153
Epoch 1: val_accuracy improved from -inf to 0.90000, saving model to /content/drive/MyDrive/Seminar2/resnet50/


50/50 [==============================] - 50s 742ms/step - loss: 4.6049 - accuracy: 0.5150 - val_loss: 5.3237 - val_accuracy: 0.9000
Epoch 2/30
50/50 [==============================] - ETA: 0s - loss: 5.9200 - accuracy: 0.6050
Epoch 2: val_accuracy improved from 0.90000 to 0.94000, saving model to /content/drive/MyDrive/Seminar2/resnet50/


50/50 [==============================] - 34s 685ms/step - loss: 5.9200 - accuracy: 0.6050 - val_loss: 5.0961 - val_accuracy: 0.9400
Epoch 3/30
50/50 [==============================] - ETA: 0s - loss: 5.5436 - accuracy: 0.6950
Epoch 3: val_accuracy improved from 0.94000 to 0.95000, saving model to /content/drive/MyDrive/Seminar2/resnet50/


50/50 [==============================] - 34s 692ms/step - loss: 5.5436 - accuracy: 0.6950 - val_loss: 5.0272 - val_accuracy: 0.9500
Epoch 4/30
50/50 [==============================] - ETA: 0s - loss: 5.8472 - accuracy: 0.6450
Epoch 4: val_accuracy did not improve from 0.95000
50/50 [==============================] - 2s 34ms/step - loss: 5.8472 - accuracy: 0.6450 - val_loss: 5.3663 - val_accuracy: 0.9400
Epoch 5/30
49/50 [============================>.] - ETA: 0s - loss: 6.3551 - accuracy: 0.6224
Epoch 5: val_accuracy did not improve from 0.95000
50/50 [==============================] - 2s 34ms/step - loss: 6.3523 - accuracy: 0.6200 - val_loss: 5.4119 - val_accuracy: 0.9300
Epoch 6/30
50/50 [==============================] - ETA: 0s - loss: 5.7687 - accuracy: 0.6700
Epoch 6: val_accuracy did not improve from 0.95000
50/50 [==============================] - 2s 34ms/step - loss: 5.7687 - accuracy: 0.6700 - val_loss: 4.7211 - val_accuracy: 0.9300
Epoch 7/30
49/50 [=========================

50/50 [==============================] - 33s 676ms/step - loss: 5.7377 - accuracy: 0.5650 - val_loss: 5.4878 - val_accuracy: 0.9600
Epoch 8/30
49/50 [============================>.] - ETA: 0s - loss: 6.3036 - accuracy: 0.6429
Epoch 8: val_accuracy did not improve from 0.96000
50/50 [==============================] - 2s 34ms/step - loss: 6.2963 - accuracy: 0.6400 - val_loss: 5.1308 - val_accuracy: 0.9600
Epoch 9/30
50/50 [==============================] - ETA: 0s - loss: 5.7948 - accuracy: 0.6550
Epoch 9: val_accuracy improved from 0.96000 to 0.97000, saving model to /content/drive/MyDrive/Seminar2/resnet50/


50/50 [==============================] - 32s 649ms/step - loss: 5.7948 - accuracy: 0.6550 - val_loss: 5.1274 - val_accuracy: 0.9700
Epoch 10/30
50/50 [==============================] - ETA: 0s - loss: 5.2902 - accuracy: 0.6900
Epoch 10: val_accuracy did not improve from 0.97000
50/50 [==============================] - 2s 34ms/step - loss: 5.2902 - accuracy: 0.6900 - val_loss: 4.6143 - val_accuracy: 0.9700
Epoch 11/30
49/50 [============================>.] - ETA: 0s - loss: 4.9563 - accuracy: 0.7500
Epoch 11: val_accuracy did not improve from 0.97000
50/50 [==============================] - 2s 34ms/step - loss: 4.9654 - accuracy: 0.7400 - val_loss: 4.4367 - val_accuracy: 0.9400
Epoch 12/30
50/50 [==============================] - ETA: 0s - loss: 5.4379 - accuracy: 0.6550
Epoch 12: val_accuracy did not improve from 0.97000
50/50 [==============================] - 2s 34ms/step - loss: 5.4379 - accuracy: 0.6550 - val_loss: 5.0866 - val_accuracy: 0.9600
Epoch 13/30
49/50 [==================

50/50 [==============================] - 34s 697ms/step - loss: 5.5492 - accuracy: 0.6600 - val_loss: 4.8917 - val_accuracy: 0.9800
Epoch 15/30
49/50 [============================>.] - ETA: 0s - loss: 5.6387 - accuracy: 0.5918
Epoch 15: val_accuracy did not improve from 0.98000
50/50 [==============================] - 2s 35ms/step - loss: 5.6335 - accuracy: 0.5950 - val_loss: 4.8399 - val_accuracy: 0.9500
Epoch 16/30
49/50 [============================>.] - ETA: 0s - loss: 5.4025 - accuracy: 0.6684
Epoch 16: val_accuracy did not improve from 0.98000
50/50 [==============================] - 2s 36ms/step - loss: 5.3990 - accuracy: 0.6700 - val_loss: 4.7454 - val_accuracy: 0.9800
Epoch 17/30
49/50 [============================>.] - ETA: 0s - loss: 5.2206 - accuracy: 0.7041
Epoch 17: val_accuracy did not improve from 0.98000
50/50 [==============================] - 2s 35ms/step - loss: 5.2279 - accuracy: 0.7000 - val_loss: 4.4881 - val_accuracy: 0.9700
Epoch 18/30
49/50 [==================

In [32]:
from sklearn import metrics

predictions = model.predict(X_test, verbose=True)
predictions = np.where(predictions > .5, 1, 0)
print(metrics.classification_report(y_test, predictions))

45/45 [==============================] - 9s 129ms/step
              precision    recall  f1-score   support

           0       0.61      0.85      0.71       378
           1       0.47      0.50      0.48       408
           2       1.00      0.47      0.64       648

   micro avg       0.66      0.58      0.61      1434
   macro avg       0.69      0.61      0.61      1434
weighted avg       0.75      0.58      0.61      1434
 samples avg       0.58      0.58      0.58      1434



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [33]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

45/45 [==============================] - 5s 92ms/step - loss: 5.2947 - accuracy: 0.6248
Test Loss: 5.294746398925781
Test Accuracy: 0.624825656414032
